In [ ]:
def get_Parallel_Pipelines(START, STOP):
    sql_query = f"""
        SELECT * FROM tcmp.cs_plrun WHERE starttime between '{START}' AND '{STOP}'
        or stoptime between '{START}' AND '{STOP}'
        ORDER BY starttime DESC
    """
    return sql_query

def get_PLRUN_Metrics(GOOD, BAD):
    sql_query = f"""
        SELECT * FROM CS_PipelineRun_Metrics where pipelinerunseq in ({GOOD}, {BAD})
    """
    return sql_query

def get_ExpensiveStatement(START, STOP):
    sql_query = f"""
        SELECT start_time, duration_microsec, records,  STATEMENT_HASH FROM M_EXPENSIVE_STATEMENTS WHERE start_time between '{START}' AND '{STOP}'
    """
    return sql_query

def get_PreviousExpensiveStatementByStatement_Hash(HASH):
    sql_query = f"""
        select STATEMENT_HASH , LAST_EXECUTION_TIMESTAMP,EXECUTION_COUNT,TOTAL_EXECUTION_TIME,MAX_EXECUTION_TIME,TOTAL_RESULT_RECORD_COUNT from _SYS_STATISTICS.HOST_SQL_PLAN_CACHE where STATEMENT_HASH in ({HASH}) order by 2 desc
    """
    return sql_query

def get_MemoryUsage (PLRUN):
    sql_query = f"""
        SELECT * FROM TCMP.OD_PIPELINE_MEM_USAGE_VW opmuv  WHERE pipelinerunseq={PLRUN} order by stage_name, workerid asc
    """
    return sql_query

def get_workSet(PERIOD):
    sql_query = f"""
        SELECT workerId, count(1) FROM CS_WorkSet WHERE periodSeq = {PERIOD} GROUP BY workerId;
    """
    return sql_query

def get_isLand(PERIOD):
    sql_query = f"""
        SELECT islandNumber, count(1) FROM CS_BUSINESSNODE WHERE periodSeq = {PERIOD} GROUP BY islandNumber ORDER BY count(1) desc;
    """
    return sql_query

def get_pipeline_runseqs_same_period_same_PU(PERIODSEQ, RUNMODE, STAGETYPESEQ, PROCESSINGUNITSEQ, TRACELEVEL, TENANTID):
    sql_query = f"""
        SELECT PIPELINERUNSEQ FROM cs_plrun 
        WHERE TENANTID = {TENANTID} 
        and PERIODSEQ = {PERIODSEQ}
        AND STAGETYPESEQ = {STAGETYPESEQ}
        AND PROCESSINGUNITSEQ = {PROCESSINGUNITSEQ}
        AND TRACELEVEL = '{TRACELEVEL}'
        AND STATUS = 'Successful'
        ORDER BY STARTTIME DESC
    """
    return sql_query

def get_pipeline_runseq(PLRUN, TENANTID):
    sql_query = f"""
        SELECT PIPELINERUNSEQ, STARTTIME, STOPTIME, RUNMODE, DESCRIPTION, STAGETYPESEQ, PERIODSEQ, PROCESSINGUNITSEQ, TRACELEVEL FROM cs_plrun where tenantid = {TENANTID} AND pipelinerunseq = {PLRUN}
    """
    return sql_query

def get_debug(TENANT, START, STOP):
    sql_query = f"""
        SELECT * FROM tcmp.cs_debug WHERE tenantid ='{TENANT}' AND DATETIME BETWEEN '{START}' AND '{STOP}'
        ORDER BY DATETIME DESC
    """
    return sql_query

def get_pipeline_stage_runtime(param):
   # Define the SQL query with a placeholder for the parameter
    sql_query = f"""
    WITH OD_PIPELINE_STAGE_RUNTIMES_VW AS(
       SELECT pr.tenantid,
              pu.NAME pu,
              p.NAME MONTH,
              pr.periodseq,
              pr.pipelinerunseq,
              pr.command stage,
              pr.runmode,
              pr.tracelevel,
              pr.status,
              pr.starttime,
              pr.stoptime
         FROM cs_pipelinerun pr, cs_period p, cs_processingunit pu
        WHERE     pr.tenantid = p.tenantid
              AND pr.periodseq = p.periodseq
              AND p.removedate = TO_DATE ('1/1/2200', 'mm/dd/yyyy')
              AND pr.tenantid = pu.tenantid
              AND pr.processingunitseq = pu.processingunitseq
              AND command = 'PipelineRun'
              AND runmode IN ('full', 'positions', 'incremental')
       UNION
       SELECT pr.tenantid,
              pu.NAME pu,
              p.NAME MONTH,
              pr.periodseq,
              pr.pipelinerunseq,
              st.NAME stage,
              pr.runmode,
              pr.tracelevel,
              ss.status,
              ss.starttime,
              ss.stoptime
         FROM cs_stagesummary ss,
              cs_stagetype st,
              cs_pipelinerun pr,
              cs_period p,
              cs_processingunit pu
        WHERE     ss.stagetypeseq = st.stagetypeseq
              AND ss.tenantid = pr.tenantid
              AND ss.pipelinerunseq = pr.pipelinerunseq
              AND pr.tenantid = p.tenantid
              AND pr.periodseq = p.periodseq
              AND p.REMOVEDATE > CURRENT_TIMESTAMP
              AND pr.tenantid = pu.tenantid
              AND pr.processingunitseq = pu.processingunitseq
              AND ss.starttime IS NOT NULL
       ORDER BY starttime DESC)
    select STAGE, STARTTIME, STOPTIME from OD_PIPELINE_STAGE_RUNTIMES_VW WHERE PIPELINERUNSEQ = {param} AND stage != 'PipelineRun' ORDER BY starttime desc;
    """
    return sql_query

def get_pipeline_allocate_runtime(param):
    sql_query = f"""
    WITH OD_FULL_PIPELINE_METRICS_VW AS (
        SELECT
                   pu.tenantid tenant,
                    pu.NAME pu,
                    pr.pipelinerunseq,
                    pr.periodseq,
                    p.NAME,
                    st.NAME stage,
                    ss.status,
                    ss.starttime,
                    ss.stoptime,
                    to_decimal(SECONDS_BETWEEN(pr.starttime,pr.stoptime)/60,10,0) elapsed_minutes,
                    to_decimal(me1.VALUE,16,0) transactions,
                    to_decimal(me2.VALUE,16,0) positions,
                    to_decimal((SELECT
                           SUM (me3.VALUE)
                       FROM cs_workersummary ws,
                            cs_metricelement me3,
                            cs_metrictype mt3
                      WHERE     pr.tenantid = ws.tenantid
                            AND pr.pipelinerunseq = ws.pipelinerunseq
                            AND ws.tenantid = me3.tenantid
                            AND ws.metricsummaryseq = me3.metricsummaryseq
                            AND mt3.metrictypeseq = me3.metrictypeseq
                            AND mt3.NAME = 'credits_created'),16,0)
                       credits
               FROM cs_pipelinerun pr,
                    cs_period p,
                    cs_processingunit pu,
                    cs_stagetype st,
                    cs_stagesummary ss,
                    cs_metricelement me1,
                    cs_metricelement me2,
                    cs_metrictype mt1,
                    cs_metrictype mt2
              WHERE     pr.tenantid = ss.tenantid
                    AND pr.pipelinerunseq = ss.pipelinerunseq
                    AND p.tenantid = pr.tenantid
                    AND p.periodseq = pr.periodseq
                    AND pu.tenantid = pr.tenantid
                    AND pu.processingunitseq =
                           IFNULL (pr.processingunitseq, pu.processingunitseq)
                    AND ss.tenantid = ss.tenantid
                    AND ss.stagetypeseq = st.stagetypeseq
                    AND ss.tenantid = me1.tenantid
                    AND ss.metricsummaryseq = me1.metricsummaryseq
                    AND ss.tenantid = me2.tenantid
                    AND ss.metricsummaryseq = me2.metricsummaryseq
                    AND mt1.metrictypeseq = me1.metrictypeseq
                    AND mt2.metrictypeseq = me2.metrictypeseq
                    AND ss.status = 'Successful'
                    AND mt1.NAME = 'unitsDone'
                    AND mt2.NAME = 'positions_assignedToPlan'
                    AND st.NAME = 'Allocate'
                    AND p.removedate = TO_DATE ('1/1/2200', 'mm/dd/yyyy')
                    AND pr.starttime IS NOT NULL
                    AND pr.status <> 'Running'
           ORDER BY starttime DESC)
        SELECT STAGE, STARTTIME, STOPTIME, TRANSACTIONS, POSITIONS, CREDITS from OD_FULL_PIPELINE_METRICS_VW WHERE pipelinerunseq = {param}"""
    return sql_query

def get_hist_stage_runtimes(plrunseqs):
    sql_query = f"""
    WITH OD_FULL_PIPELINE_METRICS_VW AS (
        SELECT
                   pu.tenantid tenant,
                    pu.NAME pu,
                    pr.pipelinerunseq,
                    pr.periodseq,
                    p.NAME,
                    st.NAME stage,
                    ss.status,
                    ss.starttime,
                    ss.stoptime,
                    to_decimal(SECONDS_BETWEEN(pr.starttime,pr.stoptime)/60,10,0) elapsed_minutes,
                    to_decimal(me1.VALUE,16,0) transactions,
                    to_decimal(me2.VALUE,16,0) positions,
                    to_decimal((SELECT
                           SUM (me3.VALUE)
                       FROM cs_workersummary ws,
                            cs_metricelement me3,
                            cs_metrictype mt3
                      WHERE     pr.tenantid = ws.tenantid
                            AND pr.pipelinerunseq = ws.pipelinerunseq
                            AND ws.tenantid = me3.tenantid
                            AND ws.metricsummaryseq = me3.metricsummaryseq
                            AND mt3.metrictypeseq = me3.metrictypeseq
                            AND mt3.NAME = 'credits_created'),16,0)
                       credits
               FROM cs_pipelinerun pr,
                    cs_period p,
                    cs_processingunit pu,
                    cs_stagetype st,
                    cs_stagesummary ss,
                    cs_metricelement me1,
                    cs_metricelement me2,
                    cs_metrictype mt1,
                    cs_metrictype mt2
              WHERE     pr.tenantid = ss.tenantid
                    AND pr.pipelinerunseq = ss.pipelinerunseq
                    AND p.tenantid = pr.tenantid
                    AND p.periodseq = pr.periodseq
                    AND pu.tenantid = pr.tenantid
                    AND pu.processingunitseq =
                           IFNULL (pr.processingunitseq, pu.processingunitseq)
                    AND ss.tenantid = ss.tenantid
                    AND ss.stagetypeseq = st.stagetypeseq
                    AND ss.tenantid = me1.tenantid
                    AND ss.metricsummaryseq = me1.metricsummaryseq
                    AND ss.tenantid = me2.tenantid
                    AND ss.metricsummaryseq = me2.metricsummaryseq
                    AND mt1.metrictypeseq = me1.metrictypeseq
                    AND mt2.metrictypeseq = me2.metrictypeseq
                    AND ss.status = 'Successful'
                    AND mt1.NAME = 'unitsDone'
                    AND mt2.NAME = 'positions_assignedToPlan'
                    AND st.NAME = 'Allocate'
                    AND p.removedate = TO_DATE ('1/1/2200', 'mm/dd/yyyy')
                    AND pr.starttime IS NOT NULL
                    AND pr.status <> 'Running'
           ORDER BY starttime DESC)
        SELECT PIPELINERUNSEQ, STAGE, STARTTIME, STOPTIME, TRANSACTIONS, POSITIONS, CREDITS from OD_FULL_PIPELINE_METRICS_VW WHERE pipelinerunseq in ({plrunseqs}) ORDER BY starttime desc"""
    return sql_query